In [1]:
import os
import numpy as np
import signal_processing as sp
import matplotlib.pyplot as plt
import time
import sklearn
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sys import getsizeof

In [2]:
# Import datasets
dataset_directory = os.path.join(os.getcwd(), "DeepSig-Dataset-2018")

# Use these to load the entire dataset into memory
fft_dataset_train = np.load(os.path.join(dataset_directory, "fft_dataset_train.npy"))
fft_dataset_val = np.load(os.path.join(dataset_directory, "fft_dataset_val.npy"))
fft_dataset_test = np.load(os.path.join(dataset_directory, "fft_dataset_test.npy"))

labels_train = np.load(os.path.join(dataset_directory, "labels_train.npy"))
labels_val = np.load(os.path.join(dataset_directory, "labels_val.npy"))
labels_test = np.load(os.path.join(dataset_directory, "labels_test.npy"))

# # Use these to use a memory map to save RAM
# fft_dataset_train = np.load(os.path.join(dataset_directory, "fft_dataset_train.npy"), mmap_mode="r+")
# fft_dataset_val = np.load(os.path.join(dataset_directory, "fft_dataset_val.npy"), mmap_mode="r+")
# fft_dataset_test = np.load(os.path.join(dataset_directory, "fft_dataset_test.npy"), mmap_mode="r+")

# labels_train = np.load(os.path.join(dataset_directory, "labels_train.npy"), mmap_mode="r+")
# labels_val = np.load(os.path.join(dataset_directory, "labels_val.npy"), mmap_mode="r+")
# labels_test = np.load(os.path.join(dataset_directory, "labels_test.npy"), mmap_mode="r+")

Super crude hyper parameter tuning by creating multiple different models

In [3]:
# Create list of models
models = []

In [4]:
# Base Model
inputs = keras.Input(shape=(1024, 1))
x = layers.Conv1D(filters=16, kernel_size=3, activation='relu')(inputs)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(filters=32, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(filters=64, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(filters=128, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(filters=256, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(filters=256, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(filters=256, kernel_size=3, activation='relu')(x)
x = layers.Flatten()(x)
x = layers.Dense(128)(x)
x = layers.Dense(128)(x)
outputs = layers.Dense(24, activation='softmax')(x)

models.append(keras.Model(inputs=inputs, outputs=outputs, name = "base_model"))

In [5]:
# Small Model
inputs = keras.Input(shape=(1024, 1))
x = layers.Conv1D(filters=16, kernel_size=3, activation='relu')(inputs)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(filters=32, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(filters=64, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(filters=128, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(filters=256, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(filters=256, kernel_size=3, activation='relu')(x)
x = layers.Flatten()(x)
x = layers.Dense(128)(x)
x = layers.Dense(128)(x)
outputs = layers.Dense(24, activation='softmax')(x)

models.append(keras.Model(inputs=inputs, outputs=outputs, name = "small_model"))

In [6]:
# Less Dense Layers
inputs = keras.Input(shape=(1024, 1))
x = layers.Conv1D(filters=16, kernel_size=3, activation='relu')(inputs)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(filters=32, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(filters=64, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(filters=128, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(filters=256, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(filters=256, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(filters=256, kernel_size=3, activation='relu')(x)
x = layers.Flatten()(x)
x = layers.Dense(128)(x)
outputs = layers.Dense(24, activation='softmax')(x)

models.append(keras.Model(inputs=inputs, outputs=outputs, name = "less_dense"))

In [7]:
# Batch normalization
inputs = keras.Input(shape=(1024, 1))
x = layers.Conv1D(filters=16, kernel_size=3, activation='relu')(inputs)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(filters=32, kernel_size=3, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(filters=64, kernel_size=3, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(filters=128, kernel_size=3, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(filters=256, kernel_size=3, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(filters=256, kernel_size=3, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(filters=256, kernel_size=3, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Flatten()(x)
x = layers.Dense(128)(x)
x = layers.Dense(128)(x)
outputs = layers.Dense(24, activation='softmax')(x)

models.append(keras.Model(inputs=inputs, outputs=outputs, name = "batch_normalization"))

In [8]:
# Double Conv Layers
inputs = keras.Input(shape=(1024, 1))
x = layers.Conv1D(filters=16, kernel_size=3, activation='relu')(inputs)
x = layers.Conv1D(filters=16, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(filters=32, kernel_size=3, activation='relu')(x)
x = layers.Conv1D(filters=32, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(filters=64, kernel_size=3, activation='relu')(x)
x = layers.Conv1D(filters=64, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(filters=128, kernel_size=3, activation='relu')(x)
x = layers.Conv1D(filters=128, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(filters=256, kernel_size=3, activation='relu')(x)
x = layers.Conv1D(filters=256, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(filters=256, kernel_size=3, activation='relu')(x)
x = layers.Conv1D(filters=256, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling1D(pool_size=2)(x)
x = layers.Conv1D(filters=256, kernel_size=3, activation='relu')(x)
x = layers.Conv1D(filters=256, kernel_size=3, activation='relu')(x)
x = layers.Flatten()(x)
x = layers.Dense(128)(x)
x = layers.Dense(128)(x)
outputs = layers.Dense(24, activation='softmax')(x)

models.append(keras.Model(inputs=inputs, outputs=outputs, name = "double_conv_layers"))

In [9]:
# Create list of optimizers to use
optimizers = ["adam", "sgd", "rmsprop"]

In [10]:
# Test
fft_dataset_train = fft_dataset_train[:100]
fft_dataset_val = fft_dataset_val[:100]
fft_dataset_test = fft_dataset_test[:100]

labels_train = labels_train[:100]
labels_val = labels_val[:100]
labels_test = labels_test[:100]

In [11]:
for optimizer in optimizers:
    for model in models:
        # Time the time it takes to train the model
        start = time.time()
        
        # Compile model
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        # Define the callbacks and save the best model to a new file
        callbacks = [keras.callbacks.ModelCheckpoint(filepath=f'models/{model.name}.keras', save_best_only=True, monitor='val_loss'), 
            keras.callbacks.EarlyStopping(monitor="val_loss", min_delta = 0.01, patience = 5, verbose = 1, restore_best_weights = True)]
        # Train model
        model.fit(fft_dataset_train, labels_train, epochs=30, batch_size = 256, validation_data = (fft_dataset_val, labels_val), callbacks=callbacks)

        # Evaluate model
        _, accuracy = model.evaluate(fft_dataset_test, labels_test)

        print("--- Model trained in %s seconds ---" % (time.time() - start))

        # Write accuracy to file
        f = open("models/accuracy_log.txt", "a")
        f.write(f"Model {model.name} evaluated an accuracy of {accuracy}.\n")
        f.write(f"Model {model.name} trained in {int(time.time() - start)} seconds.\n\n")
        f.close()

Epoch 1/30
1/1 [==============================] - 1s 1s/step - loss: 3.7699 - accuracy: 0.0600 - val_loss: 5.5929 - val_accuracy: 0.0200
Epoch 2/30
1/1 [==============================] - 0s 246ms/step - loss: 5.7461 - accuracy: 0.0500 - val_loss: 4.4640 - val_accuracy: 0.0500
Epoch 3/30
1/1 [==============================] - 0s 209ms/step - loss: 4.6950 - accuracy: 0.0600 - val_loss: 3.3205 - val_accuracy: 0.0500
Epoch 4/30
1/1 [==============================] - 0s 182ms/step - loss: 3.3984 - accuracy: 0.0600 - val_loss: 3.2574 - val_accuracy: 0.0100
Epoch 5/30
1/1 [==============================] - 0s 199ms/step - loss: 3.1771 - accuracy: 0.0400 - val_loss: 3.3193 - val_accuracy: 0.0300
Epoch 6/30
1/1 [==============================] - 0s 178ms/step - loss: 3.1429 - accuracy: 0.0500 - val_loss: 3.3000 - val_accuracy: 0.0500
Epoch 7/30
1/1 [==============================] - 0s 157ms/step - loss: 3.0905 - accuracy: 0.0700 - val_loss: 3.3266 - val_accuracy: 0.0500
Epoch 8/30
1/1 [=======

ValueError: Unknown optimizer: rsmprop. Please ensure this object is passed to the `custom_objects` argument. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.